# 벡터로서의 단어

[원문 https://iksinc.wordpress.com/tag/continuous-bag-of-words-cbow](https://iksinc.wordpress.com/tag/continuous-bag-of-words-cbow/)

벡터 공간 모델은 각 문서가 벡터로 표현되는, 정보 검색 분야에서 잘 알려저있다. 벡터 요소는 문서안에 각 단어의 가중치 또는 중요도로 표현된다. 두 문서의 유사도는 코사인 유사도 측정을 사용해 계산된다.



단어에 대한 벡터 표현을 사용한다는 아이디어가 얼마 동안 있었지만, 단어를 벡터에 매핑하는 기술인 Word Embedding 에 대한 관심이 최근 급증하고 있다.
한가지 방법은 Tomas Micolaov 의 Word2vec 알고리즘으로 많은 양의 텍스트를 사용하여 고차원(50-300)의 표현을 만들어 외부 주석이 사용하지 않는 단어 같의 관계를 캡처한다. 그러한 표현은 많은 언어적 규칙성을 포착하는 것으로 보인다. 예를 들어, 벡터 연산 vec('Paris') - vec('France') + vec('Italy') 의 결과로 vec('Rome') 에 대한 표현을 근사화 하는 벡터를 생성한다.

Word2vec 은 아래 그림과 같이 완전히 연결된 신경망으로 하나의 숨겨진 계층을 사용한다.

![screen-shot-2015-04-10-at-4-16-00-pm.png](https://iksinc.files.wordpress.com/2015/04/screen-shot-2015-04-10-at-4-16-00-pm.png)

숨겨진 계층의 뉴런들은 모두 선형 뉴런이다. 
입력 계층은 학습을 위해 어휘에 단어가 있는것과 같은 수의 뉴런을 갖도록 설정된다. 

숨겨진 계층의 크기는 결과 워드 벡터의 차원으로 설정된다. 
출력 계층의 크기는 입력 계층과 동일하다. 

그러므로, 단어 벡터를 학습하기 위한 어휘가 단어 벡터의 차원이 되는 V 개의 단어 및 N 개의 단어로 구성된다고 가정하면, 숨겨진 계층 연결에 대한 입력은 각 행이 어휘를 나타내는 V X N 크기의 행렬 WI 로 나타낼 수 있다. 
같은 방식으로 숨겨진 계층에서 출력 계층으로의 연결은 크기 N X V 행렬 WO 로 설명할 수 있다. 
이 경우, WO 행렬의 각 열은 주어진 어휘의 단어를 나타낸다.

>
예) 8개의 어휘(V)가 있고, 3개의 단어(N)의 결과를 얻겠다면, 
V X N 행렬 = ( 8, 3 ) = WI
N X V 행렬 = ( 3, 8 ) = WO


네트워크에 대한 입력은 "one hot encoding" 으로 표현된다. 하나의 입력 라인만 1, 나머지는 0으로 설정되는 것을 의미한다.


다음 문장으로 생각해보자.
``` 
“the dog saw a cat”, “the dog chased the cat”, “the cat climbed a tree” 
```
위 문장에서 코퍼스 어휘에는 8개 단어가 있다. ('the', 'dog', 'saw', 'a', 'cat', 'chased', 'climbed', 'tree')
알파벳순으로 정렬하면 각 단어를 색인으로 참조할 수 있다.

In [29]:
import numpy as np

V = np.array(['the', 'dog', 'saw', 'a', 'cat', 'chased', 'climbed', 'tree'])
V.sort()
print(V)

['a' 'cat' 'chased' 'climbed' 'dog' 'saw' 'the' 'tree']


이 예제의 경우,  신경망에 8개의 입력 뉴런과 8개의 출력 뉴런이 있다.
숨겨진 계층에는 세 개의 뉴런을 사용하기로 하자. 이것은 WI 및 WO 가 각각 8x3, 3x8 행렬임을 의미한다. 훈련이 시작되기 전에, 이 행렬은 신경망 훈련에서 평소와 같이 작은 무작위 값으로 초기화 된다.
다음 값으로 초기화 된다고 가정하자

In [32]:
WI = np.array([
    [-0.094491, 0.443977, 0.313917],
    [-0.490796, -0.229903, 0.065460],
    [-0.072921, -0.172246, -0.357751],
    [0.104514, -0.463000, 0.079367],
    [-0.226080, -0.154659, -0.038422],
    [0.406115, -0.192794, -0.441992],
    [0.181755, 0.088268, -0.277574],
    [-0.055334,0.491792, 0.263102]
])
print('WI= ', WI.shape)

WO = np.array([
    [0.023074, 0.479901, 0.432148, 0.375480, -0.364732, -0.119840, 0.266070, -0.351000],
    [-0.368008, 0.424778, -0.257104, -0.148817, 0.033922, 0.353874, -0.144942, 0.130904],
    [0.422434, 0.364503, 0.467865, -0.020302, -0.423890, -0.438777, 0.268529, -0.446787]
])

print('WO= ', WO.shape)

WI=  (8, 3)
WO=  (3, 8)


네트워크에서 'cat'과 'climbed' 단어 사이의 관계를 배우기를 원한다고 가장해보자. 즉 'cat' 이 네트워크에 입력되면, 네트워크는 'climbed'에 높은 확률을 보여야한다. 단어 임베딩 용어로 'cat'은 `Context` 'climbed' 는 `Target` 으로 표현한다.

이 경우, 입력 벡터 X 는 [ 0, 1, 0, 0, 0, 0, 0, 0 ] 로 표현될 것이다. (`cat`)
벡터의 두번째 요소만 1이라는 점을 보면, 입력된 단어가 정렬된 코퍼스 단어 목록에서 두번째 위치하는 cat 이기 때문이다.
주어진 target 단어가 'climbed' 라는 것을 고려하면, target 벡터는 [ 0, 0, 0, 1, 0, 0, 0, 0] 과 같다.


In [33]:
# cat
cat = np.array(V == 'cat', dtype=np.int)
print(cat)
climbed = np.array(V == 'climbed', dtype = np.int)
print(climbed)

[0 1 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0]


'cat'을 나타내는 입력 벡터를 사용하면 숨겨진 계층 뉴런의 출력을 다음과 같이 계산할 수 있다.
```
Ht = XtWI = [-0.490796 -0.229903 0.065460]
```

In [37]:
H = np.dot(cat, WI)
print(H)

[-0.490796 -0.229903  0.06546 ]


숨겨진 계층 뉴런의 출력 벡터가 one-hot encoding 벡터표현으로 인해 WI 행렬의 두 번째 행의 가중치를 투영한단느 것은 놀라운 일이 아니다. 따라서 숨겨진 계층 연결에 대한 입력의 기능은 기본적으로 입력 단어 벡터를 숨겨진 계층에 복사하는 것이다.

숨겨진 출력 계층에 대한 유사한 조작을 수행하면 출력 계층의 뉴런에 대한 활성화 벡터는 다음과 같이 된다.

```
HtWO = [0.100934  -0.309331  -0.122361  -0.151399   0.143463  -0.051262  -0.079686   0.112928]
```

In [38]:
output=np.dot(H, WO)
print(output)

[ 0.10093405 -0.30933086 -0.12236109 -0.15139958  0.1434624  -0.05126204
 -0.07968558  0.1129275 ]


출력 계층의 단어에 대한 확률을 생성하는것이 목표이기 때문에, k = 1, V 에 대한 P(Word k | Word context) 입력시 Context 단어와의 다음 단어의 관계를 반영하기 위해 출력 계층의 뉴런의 합계를 하나더 추가해야한다. 
Word2Vec 은 출력 계층의 뉴런 활성화 값을 확률로 변환하기 위해 softmax 함수를 사용한다.
따라서 k 번째 뉴런의 출력은 다음과 같은 식으로 계산됩니다. 여기서 activation (n)은 n 번째 출력 계층 뉴런의 활성화 값을 나타낸다.

In [40]:
def softmax(a):
    c = np.max(a)
    a_exp = np.exp(a-c)
    return (a_exp) /np.sum(a_exp)

softmax(output)

array([ 0.14307333,  0.0949255 ,  0.11444132,  0.1111659 ,  0.14928925,
        0.12287422,  0.11943087,  0.14479961])

[ 0.14307333,  0.0949255 ,  0.11444132,  **0.1111659** ,  0.14928925,  0.12287422,  0.11943087,  0.14479961]

굵게 표시된 확률은 선택한 타겟 단어가 "climbed"다. 
target 벡터 [0 0 0 1 0 0 0 0] 가 주어지면, 출력 레이어에 대한 에러 벡터는 타겟 벡터로부터 확률 벡터를 감산함으로써 쉽게 계산된다. 
에러가 알려지면 행렬 WO 및 Wi의 가중치에 backpropagation을 사용하여 업데이트 할 수 있다. 
따라서 훈련은 코퍼스와 다른 문맥 - 대상 단어 쌍을 제시함으로써 진행될 수있다. 
본질적으로, Word2vec는 단어 간의 관계를 학습하고 프로세스에서 코퍼스의 단어에 대한 벡터 표현을 개발하는 방법입니다.

## Continuous Bag of Words (CBOW) Learning

위의 설명 및 아키텍처는 단어 쌍 간의 관계를 학습하기 위한 것이다. 단어 모델의 CBOW 에서, Context 는 주어진 Target 단어에 대한 다중 단어로 표현된다. 예를 들어 'cat'과 'tree'를 컨텍스트 단어로 사용하여 'climbed' 를 대상 단어로 사용할 수 있다. 이것은 신경망 아키텍처에 대한 수정을 요구한다. 아래 표시된 수정은 숨겨진 계층 연결에 대한 입력을 C 회, context 단어수로 복제하고 숨겨진 계층 뉴런에서 C 작업으로 나누기를 추가하는 것으로 구성된다.

![](https://iksinc.files.wordpress.com/2015/04/screen-shot-2015-04-12-at-10-58-21-pm.png?w=768&h=740)

C 컨텍스트 단어를 지정하기위한 위의 구성에서, 1-V의 표현을 사용하여 코딩되는 각 단어는 숨겨진 레이어 출력이 입력시 컨텍스트 단어에 해당하는 단어 벡터의 평균임을 의미합니다. 출력 레이어는 동일하게 유지되고 교육은 위에서 설명한 방식으로 수행됩니다.

## Skip-Gram Model

스킵 그램 모델은 대상 및 컨텍스트 단어의 사용을 반대로 한다.
이 경우 타겟 단어가 입력에서 제공되고 숨겨진 레이어는 동일하게 유지되며 신경망의 출력 레이어는 선택된 컨텍스트 단어 수를 수용하기 위해 여러 번 복제된다. 
문맥 단어로 "cat"과 "tree"의 예제를 취하고 목표 단어로 "climbed"하면 스킵 - 그램 모델의 입력 벡터는 [0 0 0 1 0 0 0 0]가 되고 두 출력 계층은 각각 [0 1 0 0 0 0 0] 및 [0 0 0 0 0 1] 를 목표 벡터로 갖는다. 
하나의 확률 벡터를 생성하는 대신, 현재의 예제에 대해 2 개의 벡터가 생성됩니다. 각 출력 계층에 대한 오류 벡터는 위에 설명 된 방식으로 생성됩니다. 
그러나 모든 출력 레이어의 오류 벡터는 합산되어 역 전파를 통해 가중치를 조정합니다. 이렇게하면 각 출력 레이어에 대한 가중치 행렬 (WO)이 모두 교육을 통해 동일하게 유지됩니다.